<a href="https://colab.research.google.com/github/SylvainVerdy/NLP_playstore/blob/master/BERT_get_latent_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
!pip install transformers



In [0]:
from typing import List
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
def encode(query: str, choices: List[str]):
    """

    :param query:
    :param choices:
    :return:
    """
    inputs = [tokenizer.encode_plus(query, choice, add_special_tokens=True)
              for choice in choices]

    max_len = min(max(len(t['input_ids']) for t in inputs), max_seq_len)
    input_ids = [t['input_ids'][:max_len] +
                  [0] * (max_len - len(t['input_ids'][:max_len])) for t in inputs]
    attention_mask = [[1] * len(t['input_ids'][:max_len]) +
                      [0] * (max_len - len(t['input_ids'][:max_len])) for t in inputs]
    token_type_ids = [t['token_type_ids'][:max_len] +
                  [0] * (max_len - len(t['token_type_ids'][:max_len])) for t in inputs]

    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    token_type_ids = torch.tensor(token_type_ids)

    return input_ids, attention_mask, token_type_ids


In [0]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np

max_seq_len = 64

def hidden_states(choices, query):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  model = BertModel.from_pretrained('bert-base-uncased')
  input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)  # Batch size 1
  input_ids, attention_mask, token_type_ids = encode(query, choices)

  print(input_ids.shape)
  outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
  last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
  return last_hidden_states


In [178]:
sentences = ["test","word", "what is a gaussian model", "fall", "embeddings", "travel on my best way to improve my data"]
query = "what is a gaussian model?"
outputs = hidden_states(choices=sentences, query=query)
print(outputs.shape)


torch.Size([6, 20])
torch.Size([6, 20, 768])


In [0]:
mytensor = torch.zeros([20, 6])
np_outputs = outputs.detach().numpy()
vect_mean = np.mean(np_outputs, axis=1)

In [183]:
print(outputs.shape)
X = vect_mean
print(X.shape)
y = torch.Tensor([1,2,3,4,2,5])
print(y.shape)

torch.Size([6, 20, 768])
(6, 768)
torch.Size([6])


In [0]:
import xgboost

In [193]:
model = xgboost.sklearn.XGBClassifier()
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [196]:
output = hidden_states(choices=["fall"], query=query)
np_output = output.detach().numpy()
np_output = np.mean(np_output, axis=1)
y_pred = model.predict(np_output)
print(y_pred)

torch.Size([1, 12])
[3.]
